In [7]:
import numpy as np
import matplotlib.pyplot as plt
import sys

In [359]:
def distance_calc(obj, centroids):
    
    dist = np.array([])
    
    for i in range(len(obj)):
        for j in range(len(centroids)):
            
            dist = np.append(dist, np.sqrt((obj[i][0]-centroids[j][0])**2 + (obj[i][1]-centroids[j][1])**2))
            
    dist = np.reshape(dist, (len(centroids),len(obj)))
    
    return dist


def choose_cluster(dist):

    clusters = np.array([])
    min_dist = 0.
    group = 0
               
    for i in range(dist.shape[1]):
        for j in range(dist.shape[0]):
                        
            if j == 0:
                min_dist = dist[j][i]
                group = j
                
            elif dist[j][i] < min_dist:
                min_dist = dist[j][i]
                group = j
                
        clusters = np.append(clusters, group)
        
    return clusters


def recalc_centroids(obj, clusters):
    
    new_centroids = np.array([])
    
    for i in range(len(np.unique(clusters))):
        new_centroids = np.append(new_centroids, np.average(obj[np.where(clusters==i)], axis=0))
    
    return new_centroids.reshape(2,2)


In [362]:
obj = np.array([[1,1],[1,2],[2,1],[3,2],[3,4],[4,4]])
centroids = np.array([[3,1],[1,4]])

distances = distance_calc(obj, centroids)
print distances
clusters = choose_cluster(distances)
new_centroids = recalc_centroids(obj, clusters)


[[2.         3.         2.23606798 2.         1.         3.16227766]
 [1.         2.82842712 3.         2.         3.16227766 3.        ]]
